In [ ]:
from google.colab import drive
import pandas as pd
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline

# Шаг 1: Монтирование Google Drive
drive.mount('/content/drive')  # Монтируем Google Drive

# Шаг 2: Загрузка файла
file_path = '/content/drive/MyDrive/Colab Notebooks/contract.csv'  # Путь к вашему файлу
data = pd.read_csv(file_path)  # Загрузка данных из CSV

# Преобразование столбца InsuranceAmount в числовой формат
data['InsuranceAmount'] = pd.to_numeric(data['InsuranceAmount'], errors='coerce')

print("Данные после импорта:")
print(data.head())  # Вывод первых нескольких строк таблицы

# Шаг 3: Определение идентификаторов строк с максимальным и минимальным значением
max_id = data.loc[data['InsuranceAmount'].idxmax(), 'ContractID']
min_id = data.loc[data['InsuranceAmount'].idxmin(), 'ContractID']

print(f"Идентификатор строки с максимальным значением InsuranceAmount: {max_id}")
print(f"Идентификатор строки с минимальным значением InsuranceAmount: {min_id}")

# Определение минимального коэффициента ставки вручную
min_rate_manual = data['Rate'].min()
print(f"Минимальный коэффициент ставки (ручной расчет): {min_rate_manual}")

# Шаг 4: Загрузка модели
model_name = "google/tapas-base-finetuned-wtq"  # Выбор модели
model = AutoModelForTableQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Инициализация pipeline
table_qa_pipeline = pipeline("table-question-answering", model=model, tokenizer=tokenizer)

# Шаг 2: Функция для обработки запросов
def query_data(query, data):
    if "минимальный коэффициент ставки" in query:
        return data['Rate'].min()
    elif "максимальное значение страховой суммы" in query:
        return data['InsuranceAmount'].max()
    elif "минимальное значение в столбце 'Rate'" in query:
        return data['Rate'].min()
    elif "максимальное значение в столбце 'Rate'" in query:
        return data['Rate'].max()
    elif "сколько уникальных клиентов" in query:  # Изменено условие
        return data['ClientID'].nunique()
    elif "сколько всего контрактов" in query:
        return len(data)
    else:
        return "Неизвестный запрос."

# Шаг 3: Примеры запросов
queries = [
    "Какой минимальный коэффициент ставки?",
    "Какое максимальное значение страховой суммы?",
    "Какое минимальное значение в столбце 'Rate'?",
    "Какое максимальное значение в столбце 'Rate'?",
    "сколько уникальных клиентов 'ClientID'?"  # Изменён запрос
]

# Шаг 4: Обработка запросов
for q in queries:
    answer = query_data(q, data)
    print(f"Запрос: {q}\nОтвет: {answer}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Данные после импорта:
   ContractID ContractDate  InsuranceAmount  Rate  AgentID  ClientID  \
0          77   2024-01-15         150000.0  2.75        1         1   
1          78   2024-02-20         200000.0  3.00        2         2   
2          79   2024-03-05         250000.0  2.50        3         3   
3          80   2024-04-10         300000.0  2.25        3         4   
4          84   2024-10-18         150000.0  5.50        1         1   

   InsuranceTypeID  BranchID  
0                1         1  
1                2         2  
2                3         3  
3                4         1  
4                2         1  
Идентификатор строки с максимальным значением InsuranceAmount: 80
Идентификатор строки с минимальным значением InsuranceAmount: 77
Минимальный коэффициент ставки (ручной расчет): 2.25
Запрос: Какой минимальный коэффициент ставки?
